# Импортируем необходимые библиотеки

In [2]:
import json
from datetime import datetime
from datetime import timedelta
import time
import requests
import re
import pandas as pd
import numpy as np
import os
import yaml
import sql_handler
import postgresql_handler
import clickhouse_handler
import docker_service


# Опередляем объём импорта и необходимые объекты импорта

In [3]:
has_errors = False
table_settings_mapper = {
    'deal': 'deal_fields',
    'contact': 'contact_fields',
    'company': 'company_fields',
    'lead': 'lead_fields',
}

database_import_mapper = {
    'MySQL': sql_handler,
    'PostgreSQL': postgresql_handler,
    'ClickHouse': clickhouse_handler
}

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

database_type = config.get('database_type')

if database_type in database_import_mapper:
    handler = database_import_mapper[database_type]
else:
    print('Unsupported database type')
    # docker_service.close()


columns = handler.get_columns_and_types_sql(
    config['table_names']['deal'], **config['db'])
set_of_columns = set([column['name'] for column in columns])
for column in config['deal_fields']:
    if config['deal_fields'][column] not in set_of_columns:
        if has_errors == False:
            has_errors = True
            list_of_errors = []
        list_of_errors.append(
            f"Column {column} not found in table {config['table_names']['deal']}")
if has_errors:
    print(*list_of_errors, '\n')
    print('Please, check your config.yaml file and database. After that, restart the container.')
    # docker_service.close()


Column bla_bla not found in table deals_b24 

Please, check your config.yaml file and database. After that, restart the container.


In [20]:

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
handler.test_connection(**config['db'])

NameError: name 'handler' is not defined

In [2]:
import sys
!{sys.executable} -m pip install black